In [4]:
import ammonite.ops._
val jarString = "target/scala-3.0.2/euclid-scala-assembly-0.1.0-SNAPSHOT.jar"
val path = ammonite.ops.pwd / ammonite.ops.RelPath(jarString)
interp.load.cp(path)

import ammonite.ops._

jarString: String = "target/scala-3.0.2/euclid-scala-assembly-0.1.0-SNAPSHOT.jar"
path: Path = /Users/michaelmccoy/Projects/euclid-scala/notebooks/target/scala-3.0.2/euclid-scala-assembly-0.1.0-SNAPSHOT.jar

In [5]:
def makeExponents(order: Int, l: List[Int]): List[(Int, Int)] = {
    l.zipWithIndex.map(t => (t._2, t._1)) :::((order, 1) :: Nil)
}

defined function makeExponents

In [6]:
import com.mbmccoy.euclid.polynomial.Prime
import com.mbmccoy.euclid.polynomial.FiniteField

val prime = Prime.next(2)
val order = 63
// Specifying exponents creates the field significantly faster
val exponents: Iterable[(Int, Int)] = makeExponents(
    order, 
    1 :: 1 ::1 :: 1 :: 1 :: 0 :: 0 :: 0 :: 1 :: 1 :: 0 :: 1 :: 0 :: 0 :: 0 :: 1 :: 1 :: 1 :: 0 :: 0 ::0 :: 0 :: 1 :: 1:: 1 :: Nil
    ) 
val cf = FiniteField(prime, order, exponents)
println(cf.conwayPolynomial)

1 + X + X^2 + X^3 + X^4 + X^8 + X^9 + X^11 + X^15 + X^16 + X^17 + X^22 + X^23 + X^24 + X^63


import com.mbmccoy.euclid.polynomial.Prime

import com.mbmccoy.euclid.polynomial.FiniteField


prime: Prime = Prime(value = 2)
order: Int = 63
exponents: Iterable[Tuple2[Int, Int]] = List(
  (0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 0),
  (6, 0),
  (7, 0),
  (8, 1),
  (9, 1),
  (10, 0),
  (11, 1),
  (12, 0),
  (13, 0),
  (14, 0),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 0),
  (19, 0),
  (20, 0),
  (21, 0),
  (22, 1),
  (23, 1),
  (24, 1),
  (63, 1)
)
cf: FiniteField = com.mbmccoy.euclid.polynomial.FiniteField@341a800d

In [7]:

val prime = Prime.next(5)
val order = 2
val exponents = makeExponents(
    order,
    2::7::1::Nil
)
val cf = FiniteField(prime, order, exponents)
println(cf.conwayPolynomial)


2 + 2X + X^2


prime: Prime = Prime(value = 5)
order: Int = 2
exponents: List[Tuple2[Int, Int]] = List((0, 2), (1, 7), (2, 1), (2, 1))
cf: FiniteField = com.mbmccoy.euclid.polynomial.FiniteField@3f1b9c36

In [8]:
val prime = Prime.next(65537)
val order = 1
val cf = FiniteField(prime, order)
cf.conwayPolynomial

Loading Conway polynomial database...
Done.


: 